I chose to make a model that takes in 5 years of stock data and will decide whether to buy, sell, or hold that stock position.

I am comparing a baseline Linear NN with two more advanced models to see whether predictions and accuracy improve with more complexity.

In [1]:
%pip install pandas numpy yfinance scikit-learn torch matplotlib


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# Choose Stock and Download Data

In [3]:
stock_ticker = 'TSLA'  # Stock Ticker to predict buy, sell, or hold
sp500_ticker = '^GSPC'  # S&P 500 index

start_date = '2015-01-01'
end_date = datetime.date.today().strftime('%Y-%m-%d')
# Fetch the data using yfinance
stock_data = yf.download(stock_ticker, start=start_date, end=end_date)
sp500_data = yf.download(sp500_ticker, start=start_date, end=end_date)

data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
data['SP500_Close'] = sp500_data['Close']

data.columns = data.columns.get_level_values(0)

data.ffill(inplace=True)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Create Buy, Sell, and Hold Labels

In [4]:
THRESHOLD = 0.003 #hold threshold if open and close are within .5% of each other


def label_data(row):
    change = abs((row['Close'] - row['Open']) / row['Open'])
    if change < THRESHOLD:
        return 0
    elif row['Open'] < row['Close']:
        return 1
    else:
        return -1

data['Label']  = data.apply(label_data, axis=1)
data['Next_Open'] = data['Open'].shift(-1)
data['Next_Label'] = data['Label'].shift(-1)

data.dropna(inplace=True)

# Setup Dataloaders

In [5]:
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'SP500_Close']
scaler = StandardScaler()
X = scaler.fit_transform(data[features])
y = data['Label'].values

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)


batch_size = 1024
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)


# Baseline Linear Neural Network Training

In [6]:
class BaseLineNN(nn.Module):
    def __init__(self,  input_dim, output_dim):
        super(BaseLineNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.fc2 = nn.Linear(16, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_dim = len(features)
output_dim = 3
baseline_model = BaseLineNN(input_dim, output_dim)

baseline_criterion = nn.CrossEntropyLoss()
baseline_optimizer = optim.SGD(baseline_model.parameters(), lr=0.01)

In [7]:
num_epochs = 1000
best_baseline_loss = float("inf")
baseline_model_path = "best_baseline_model.pth"

for epoch in range(num_epochs):
    baseline_model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        baseline_optimizer.zero_grad()
        outputs = baseline_model(inputs)
        loss = baseline_criterion(outputs, labels + 1)
        loss.backward()
        baseline_optimizer.step()
        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == (labels + 1)).sum().item()
        total_train += labels.size(0)

    train_accuracy = correct_train / total_train

    baseline_model.eval()
    total_val_loss = 0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = baseline_model(inputs)
            val_loss = baseline_criterion(outputs, labels + 1)
            total_val_loss += val_loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_test += (predicted == (labels + 1)).sum().item()
            total_test += labels.size(0)

    val_accuracy = correct_test / total_test
    avg_train_loss = total_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(test_loader)

    if avg_val_loss < best_baseline_loss:
        best_baseline_loss = avg_val_loss
        torch.save(baseline_model.state_dict(), baseline_model_path)

    if epoch % 10 == 0:
        print(f"Baseline Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4%} | "
              f"Test Loss: {avg_val_loss:.4f} | Test Acc: {val_accuracy:.4%}")

print(f"\nBest baseline model saved to: {baseline_model_path} with Test Loss: {best_baseline_loss:.4f}")

Baseline Epoch [1/1000] | Train Loss: 1.1806 | Train Acc: 28.0079% | Test Loss: 1.1738 | Test Acc: 28.7968%
Baseline Epoch [11/1000] | Train Loss: 1.1140 | Train Acc: 30.1282% | Test Loss: 1.1099 | Test Acc: 31.3609%
Baseline Epoch [21/1000] | Train Loss: 1.0724 | Train Acc: 45.1677% | Test Loss: 1.0696 | Test Acc: 45.1677%
Baseline Epoch [31/1000] | Train Loss: 1.0447 | Train Acc: 45.1183% | Test Loss: 1.0427 | Test Acc: 45.1677%
Baseline Epoch [41/1000] | Train Loss: 1.0253 | Train Acc: 45.1183% | Test Loss: 1.0238 | Test Acc: 44.9704%
Baseline Epoch [51/1000] | Train Loss: 1.0113 | Train Acc: 45.1677% | Test Loss: 1.0100 | Test Acc: 44.7732%
Baseline Epoch [61/1000] | Train Loss: 1.0009 | Train Acc: 45.1677% | Test Loss: 0.9997 | Test Acc: 44.5759%
Baseline Epoch [71/1000] | Train Loss: 0.9929 | Train Acc: 45.1183% | Test Loss: 0.9918 | Test Acc: 44.5759%
Baseline Epoch [81/1000] | Train Loss: 0.9868 | Train Acc: 45.2170% | Test Loss: 0.9856 | Test Acc: 44.9704%
Baseline Epoch [91/1

# Medium Neural Network Training

In [8]:
class MediumClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MediumClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # No activation (CrossEntropyLoss expects raw logits)
        return x

# Initialize model
input_dim = len(features)
output_dim = 3  # Buy, Sell, Hold
mediumClassifier = MediumClassifier(input_dim, output_dim)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(mediumClassifier.parameters(), lr=0.001)

In [9]:
# Training Loop
num_epochs = 1000
best_val_loss = float("inf")  # Track lowest validation loss
best_model_path = "best_stock_model.pth"  # Save model path

for epoch in range(num_epochs):
    mediumClassifier.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    # Training loop
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = mediumClassifier(inputs)
        loss = criterion(outputs, labels + 1)  # Shift labels (-1,0,1) → (0,1,2)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Compute training accuracy
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == (labels + 1)).sum().item()
        total_train += labels.size(0)

    train_accuracy = correct_train / total_train

    # Validation loop
    mediumClassifier.eval()
    total_val_loss = 0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = mediumClassifier(inputs)
            val_loss = criterion(outputs, labels + 1)
            total_val_loss += val_loss.item()

            # Compute test accuracy
            _, predicted = torch.max(outputs, 1)
            correct_test += (predicted == (labels + 1)).sum().item()
            total_test += labels.size(0)

    val_accuracy = correct_test / total_test
    avg_train_loss = total_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(test_loader)

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(mediumClassifier.state_dict(), best_model_path)

    # Print epoch summary
    if epoch % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] | "
            f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4%} | "
            f"Test Loss: {avg_val_loss:.4f} | Test Acc: {val_accuracy:.4%}")

print(f"\nBest model saved to: {best_model_path} with Test Loss: {best_val_loss:.4f}")

Epoch [1/1000] | Train Loss: 1.1029 | Train Acc: 34.1716% | Test Loss: 1.0914 | Test Acc: 43.1953%
Epoch [11/1000] | Train Loss: 0.9791 | Train Acc: 44.6746% | Test Loss: 0.9737 | Test Acc: 45.1677%
Epoch [21/1000] | Train Loss: 0.9556 | Train Acc: 46.4990% | Test Loss: 0.9570 | Test Acc: 42.9980%
Epoch [31/1000] | Train Loss: 0.9497 | Train Acc: 46.9921% | Test Loss: 0.9529 | Test Acc: 45.1677%
Epoch [41/1000] | Train Loss: 0.9470 | Train Acc: 48.0769% | Test Loss: 0.9501 | Test Acc: 45.5621%
Epoch [51/1000] | Train Loss: 0.9446 | Train Acc: 48.4714% | Test Loss: 0.9489 | Test Acc: 46.5483%
Epoch [61/1000] | Train Loss: 0.9427 | Train Acc: 48.5700% | Test Loss: 0.9469 | Test Acc: 45.9566%
Epoch [71/1000] | Train Loss: 0.9405 | Train Acc: 48.6686% | Test Loss: 0.9449 | Test Acc: 46.5483%
Epoch [81/1000] | Train Loss: 0.9381 | Train Acc: 49.0631% | Test Loss: 0.9428 | Test Acc: 47.9290%
Epoch [91/1000] | Train Loss: 0.9354 | Train Acc: 49.8028% | Test Loss: 0.9399 | Test Acc: 48.7179%
E

# Advanced Neural Network Training

In [10]:
class ImprovedStockClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ImprovedStockClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.3)

        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)

        self.fc3 = nn.Linear(64, 32)
        self.bn3 = nn.BatchNorm1d(32)
        self.dropout3 = nn.Dropout(0.2)

        self.fc4 = nn.Linear(32, output_dim)

        self.activation = nn.LeakyReLU(negative_slope=0.01)

    def forward(self, x):
        x = self.activation(self.bn1(self.fc1(x)))
        x = self.dropout1(x)

        x = self.activation(self.bn2(self.fc2(x)))
        x = self.dropout2(x)

        x = self.activation(self.bn3(self.fc3(x)))
        x = self.dropout3(x)

        x = self.fc4(x)
        return x

improved_model = ImprovedStockClassifier(input_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(improved_model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

In [11]:
num_epochs = 1000
best_improved_loss = float("inf")
improved_model_path = "best_improved_model.pth"

for epoch in range(num_epochs):
    improved_model.train()
    total_loss = 0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = improved_model(inputs)
        loss = criterion(outputs, labels + 1)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == (labels + 1)).sum().item()
        total_train += labels.size(0)

    train_accuracy = correct_train / total_train

    improved_model.eval()
    total_val_loss = 0
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = improved_model(inputs)
            val_loss = criterion(outputs, labels + 1)
            total_val_loss += val_loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_test += (predicted == (labels + 1)).sum().item()
            total_test += labels.size(0)

    val_accuracy = correct_test / total_test
    avg_train_loss = total_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(test_loader)

    if avg_val_loss < best_improved_loss:
        best_improved_loss = avg_val_loss
        torch.save(improved_model.state_dict(), improved_model_path)

    scheduler.step()

    if epoch % 10 == 0:
        print(f"Improved Model - Epoch [{epoch+1}/{num_epochs}] | "
              f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.4%} | "
              f"Test Loss: {avg_val_loss:.4f} | Test Acc: {val_accuracy:.4%}")

print(f"\nBest improved model saved to: {improved_model_path} with Test Loss: {best_improved_loss:.4f}")

Improved Model - Epoch [1/1000] | Train Loss: 1.2709 | Train Acc: 18.6391% | Test Loss: 1.1662 | Test Acc: 10.0592%
Improved Model - Epoch [11/1000] | Train Loss: 1.0566 | Train Acc: 41.7160% | Test Loss: 1.0807 | Test Acc: 39.4477%
Improved Model - Epoch [21/1000] | Train Loss: 0.9846 | Train Acc: 49.0631% | Test Loss: 0.9780 | Test Acc: 48.9152%
Improved Model - Epoch [31/1000] | Train Loss: 0.9236 | Train Acc: 53.7475% | Test Loss: 0.9197 | Test Acc: 55.6213%
Improved Model - Epoch [41/1000] | Train Loss: 0.8581 | Train Acc: 56.5089% | Test Loss: 0.8531 | Test Acc: 60.1578%
Improved Model - Epoch [51/1000] | Train Loss: 0.8086 | Train Acc: 59.3688% | Test Loss: 0.7570 | Test Acc: 63.5108%
Improved Model - Epoch [61/1000] | Train Loss: 0.7805 | Train Acc: 60.1578% | Test Loss: 0.7163 | Test Acc: 64.4970%
Improved Model - Epoch [71/1000] | Train Loss: 0.7581 | Train Acc: 61.4398% | Test Loss: 0.6867 | Test Acc: 67.8501%
Improved Model - Epoch [81/1000] | Train Loss: 0.7397 | Train Acc

# Compare and Evalutate both models

In [12]:
def evaluate_model(model, test_loader, name="Model"):
    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.numpy())
            actuals.extend((labels + 1).numpy())  # Shift back from (-1,0,1) to (0,1,2)

    print(f"{name} Classification Report:")
    print(classification_report(actuals, predictions, target_names=['Sell', 'Hold', 'Buy']))
    return predictions, actuals

# Evaluate both models
predictions_baseline, actuals_baseline = evaluate_model(baseline_model, test_loader, name="Baseline NN")
predictions_baseline, actuals_baseline = evaluate_model(mediumClassifier, test_loader, name="Baseline NN")
predictions_advanced, actuals_advanced = evaluate_model(improved_model, test_loader, name="Advanced NN")

Baseline NN Classification Report:
              precision    recall  f1-score   support

        Sell       0.45      0.58      0.51       229
        Hold       0.00      0.00      0.00        51
         Buy       0.49      0.46      0.48       227

    accuracy                           0.47       507
   macro avg       0.31      0.35      0.33       507
weighted avg       0.42      0.47      0.44       507

Baseline NN Classification Report:
              precision    recall  f1-score   support

        Sell       0.85      0.91      0.88       229
        Hold       0.71      0.10      0.17        51
         Buy       0.83      0.92      0.87       227

    accuracy                           0.83       507
   macro avg       0.80      0.64      0.64       507
weighted avg       0.82      0.83      0.80       507

Advanced NN Classification Report:
              precision    recall  f1-score   support

        Sell       0.90      0.98      0.94       229
        Hold       1.00 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [13]:
latest_data = stock_data.iloc[-1][['Open', 'High', 'Low', 'Close', 'Volume']]
latest_sp500 = sp500_data.iloc[-1]['Close']

latest_features = np.array([latest_data['Open'], latest_data['High'], latest_data['Low'], latest_data['Close'], latest_data['Volume'], latest_sp500])
latest_features = latest_features.reshape(1, -1)
latest_features_scaled = scaler.transform(latest_features)
latest_tensor = torch.tensor(latest_features_scaled, dtype=torch.float32)

baseline_model.eval()
with torch.no_grad():
    output = baseline_model(latest_tensor)
    _, predicted_class = torch.max(output, 1)

label_map = {0: "Sell", 1: "Hold", 2: "Buy"}
predicted_label = label_map[predicted_class.item()]

print(f"Baseline model: Today's recommended action for {stock_ticker}: {predicted_label}")

Baseline model: Today's recommended action for TSLA: Buy


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [14]:
latest_data = stock_data.iloc[-1][['Open', 'High', 'Low', 'Close', 'Volume']]
latest_sp500 = sp500_data.iloc[-1]['Close']

latest_features = np.array([latest_data['Open'], latest_data['High'], latest_data['Low'], latest_data['Close'], latest_data['Volume'], latest_sp500])
latest_features = latest_features.reshape(1, -1)
latest_features_scaled = scaler.transform(latest_features)
latest_tensor = torch.tensor(latest_features_scaled, dtype=torch.float32)

mediumClassifier.eval()
with torch.no_grad():
    output = mediumClassifier(latest_tensor)
    _, predicted_class = torch.max(output, 1)

label_map = {0: "Sell", 1: "Hold", 2: "Buy"}
predicted_label = label_map[predicted_class.item()]

print(f"Medium model: Today's recommended action for {stock_ticker}: {predicted_label}")

Medium model: Today's recommended action for TSLA: Buy


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [15]:
latest_data = stock_data.iloc[-1][['Open', 'High', 'Low', 'Close', 'Volume']]
latest_sp500 = sp500_data.iloc[-1]['Close']

latest_features = np.array([latest_data['Open'], latest_data['High'], latest_data['Low'], latest_data['Close'], latest_data['Volume'], latest_sp500])
latest_features = latest_features.reshape(1, -1)
latest_features_scaled = scaler.transform(latest_features)
latest_tensor = torch.tensor(latest_features_scaled, dtype=torch.float32)

improved_model.eval()
with torch.no_grad():
    output = improved_model(latest_tensor)
    _, predicted_class = torch.max(output, 1)

label_map = {0: "Sell", 1: "Hold", 2: "Buy"}
predicted_label = label_map[predicted_class.item()]

print(f"Improved model: Today's recommended action for {stock_ticker}: {predicted_label}")

Improved model: Today's recommended action for TSLA: Buy


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
